In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 17995956
paper_name = 'schmidlin_kennedy_2008' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/TableS2.xlsx', sheet_name='Sheet1')

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 100 x 5


In [7]:
original_data.tail()

,Unnamed: 0,ORF,Unnamed: 2,Notes,Quality control issues
95,96,YPR073C,LTP1,Not different from wild-type mating,NaN
96,97,YPR083W,MDM36,Not different from wild-type mating,NaN
97,98,YPR100W,MRPL51,Not different from wild-type mating,NaN
98,99,YPR170C,Dubious ORF,Diploid mates as a,NaN
99,100,YCL016C,DCC1,Diploid mates as both a and α,NaN


In [9]:
original_data['orf'] = original_data['ORF'].astype(str)

In [10]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [11]:
# Mutants marked in green in DOC file were identified in the original screen, except for YCL016C
hits = ['YHR191C','YIR005W','YLR193C','YMR078C','YOR305W','YPR170C']
original_data = original_data.loc[original_data['orf'].isin(hits),:].copy()

In [12]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [13]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [Unnamed: 0, ORF, Unnamed: 2, Notes, Quality control issues, orf]
Index: []


In [14]:
original_data.set_index('orf', inplace=True)
original_data.shape

(6, 5)

In [15]:
original_data.head(n=10)

,Unnamed: 0,ORF,Unnamed: 2,Notes,Quality control issues
orf,,,,,
YHR191C,46,YHR191C,CTF8,Diploid mates as both a and alpha,NaN
YIR005W,54,YIR005W,IST3,Diploid mates as alpha,NaN
YLR193C,68,YLR193C,Unchar ORF,Diploid mates as both a and alpha,NaN
YMR078C,71,YMR078C,CTF18,Diploid mates as both a and alpha,NaN
YOR305W,87,YOR305W,Unchar ORF,Diploid mates as both a and alpha,NaN
YPR170C,99,YPR170C,Dubious ORF,Diploid mates as a,NaN


In [16]:
original_data['a'] = 0
original_data['alpha'] = 0

In [17]:
original_data['a'] = original_data['Notes'].apply(lambda x: 1 if (x == 'Diploid mates as both a and alpha') or (x == 'Diploid mates as a') else 0)
original_data['alpha'] = original_data['Notes'].apply(lambda x: 1 if (x == 'Diploid mates as both a and alpha') or (x == 'Diploid mates as alpha') else 0)

In [18]:
original_data = original_data[['a','alpha']].copy()

In [19]:
original_data = original_data.groupby(original_data.index).mean()

In [20]:
original_data.shape

(6, 2)

# Prepare the final dataset

In [21]:
data = original_data.copy()

In [22]:
dataset_ids = [16603,16707]
datasets = datasets.reindex(index=dataset_ids)

In [23]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [24]:
data.head()

dataset_id,16603,16707
data_type,value,value
orf,,
YHR191C,1,1
YIR005W,0,1
YLR193C,1,1
YMR078C,1,1
YOR305W,1,1


## Subset to the genes currently in SGD

In [25]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [26]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,16603,16707
,data_type,value,value
gene_id,orf,,
1159,YHR191C,1,1
1361,YIR005W,0,1
3882,YLR193C,1,1
4372,YMR078C,1,1
5517,YOR305W,1,1


# Normalize

In [27]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [28]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [29]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,16603,16707,16603,16707
,data_type,value,value,valuez,valuez
gene_id,orf,,,,
1159,YHR191C,1,1,30.179463,30.179463
1361,YIR005W,0,1,0.000000,30.179463
3882,YLR193C,1,1,30.179463,30.179463
4372,YMR078C,1,1,30.179463,30.179463
5517,YOR305W,1,1,30.179463,30.179463


# Print out

In [30]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [31]:
from IO.save_data_to_db3 import *

In [32]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/2 [00:00<?, ?it/s]

Deleting all datasets for PMID 17995956...
Inserting the new data...


100%|██████████| 2/2 [00:00<00:00, 15.62it/s]

Updating the data_modified_on field...
